In [1]:
# import Dependancies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [4]:
# Create a set of random latitube and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats,lngs)

# Add the latitudes and longitudes to a list.
#lat_lngs
coordinates = list(lat_lngs)
#coordinates

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
#len(cities)

# Create an empty list to hold weather data.
city_data = []
# Print the beginning Data Retrieval
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
Total_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    #Group cities in sets of 50 for logging purposes.
    if ((i % 50 == 0) and (i >= 50)):
        set_count += 1
        record_count = 1
        time.sleep(60)
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record ({Total_count}){record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    Total_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")



Beginning Data Retrieval     
-----------------------------
Processing Record (1)1 of Set 1 | albany
Processing Record (2)2 of Set 1 | katobu
Processing Record (3)3 of Set 1 | ribeira grande
Processing Record (4)4 of Set 1 | hobart
Processing Record (5)5 of Set 1 | andros town
Processing Record (6)6 of Set 1 | salmon arm
Processing Record (7)7 of Set 1 | bengkulu
Processing Record (8)8 of Set 1 | khatanga
Processing Record (9)9 of Set 1 | saldanha
Processing Record (10)10 of Set 1 | vaini
Processing Record (11)11 of Set 1 | namibe
Processing Record (12)12 of Set 1 | busselton
Processing Record (13)13 of Set 1 | santa cruz
Processing Record (14)14 of Set 1 | doctor pedro p. pena
City not found. Skipping...
Processing Record (15)15 of Set 1 | jamestown
Processing Record (16)16 of Set 1 | tilichiki
Processing Record (17)17 of Set 1 | ushuaia
Processing Record (18)18 of Set 1 | mandalgovi
Processing Record (19)19 of Set 1 | hermanus
Processing Record (20)20 of Set 1 | provideniya
Processin

Processing Record (172)22 of Set 4 | georgetown
Processing Record (173)23 of Set 4 | niquelandia
Processing Record (174)24 of Set 4 | utrecht
Processing Record (175)25 of Set 4 | ancud
Processing Record (176)26 of Set 4 | buala
Processing Record (177)27 of Set 4 | lucapa
Processing Record (178)28 of Set 4 | brusturoasa
Processing Record (179)29 of Set 4 | aitape
Processing Record (180)30 of Set 4 | santa rita
Processing Record (181)31 of Set 4 | belushya guba
City not found. Skipping...
Processing Record (182)32 of Set 4 | oranjestad
Processing Record (183)33 of Set 4 | mananjary
Processing Record (184)34 of Set 4 | caravelas
Processing Record (185)35 of Set 4 | karaul
City not found. Skipping...
Processing Record (186)36 of Set 4 | tshane
Processing Record (187)37 of Set 4 | lavrentiya
Processing Record (188)38 of Set 4 | san policarpo
Processing Record (189)39 of Set 4 | upernavik
Processing Record (190)40 of Set 4 | coihaique
Processing Record (191)41 of Set 4 | broome
Processing Re

Processing Record (341)41 of Set 7 | darnah
Processing Record (342)42 of Set 7 | vigrestad
Processing Record (343)43 of Set 7 | quatre cocos
Processing Record (344)44 of Set 7 | mushie
Processing Record (345)45 of Set 7 | beloha
Processing Record (346)46 of Set 7 | ulladulla
Processing Record (347)47 of Set 7 | yatou
Processing Record (348)48 of Set 7 | constitucion
Processing Record (349)49 of Set 7 | bambanglipuro
Processing Record (350)50 of Set 7 | ulaanbaatar
Processing Record (351)1 of Set 8 | huainan
Processing Record (352)2 of Set 8 | sandnessjoen
Processing Record (353)3 of Set 8 | kailua
Processing Record (354)4 of Set 8 | jieshi
Processing Record (355)5 of Set 8 | margate
Processing Record (356)6 of Set 8 | tezu
Processing Record (357)7 of Set 8 | taramangalam
Processing Record (358)8 of Set 8 | veraval
Processing Record (359)9 of Set 8 | ponta do sol
Processing Record (360)10 of Set 8 | liwale
Processing Record (361)11 of Set 8 | mont-dore
Processing Record (362)12 of Set 8

Processing Record (509)9 of Set 11 | leh
Processing Record (510)10 of Set 11 | port hedland
Processing Record (511)11 of Set 11 | port-gentil
Processing Record (512)12 of Set 11 | geraldton
Processing Record (513)13 of Set 11 | katsuura
Processing Record (514)14 of Set 11 | plainview
Processing Record (515)15 of Set 11 | mogadishu
Processing Record (516)16 of Set 11 | vila franca do campo
Processing Record (517)17 of Set 11 | san joaquin
Processing Record (518)18 of Set 11 | airai
Processing Record (519)19 of Set 11 | san quintin
Processing Record (520)20 of Set 11 | urusha
Processing Record (521)21 of Set 11 | itarema
Processing Record (522)22 of Set 11 | pangnirtung
Processing Record (523)23 of Set 11 | bitam
Processing Record (524)24 of Set 11 | tyumentsevo
Processing Record (525)25 of Set 11 | svetlaya
Processing Record (526)26 of Set 11 | abu zabad
Processing Record (527)27 of Set 11 | prince rupert
Processing Record (528)28 of Set 11 | lompoc
Processing Record (529)29 of Set 11 |

In [5]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,18.30,72,21,1.74,US,2022-01-31 13:42:44
1,Katobu,-4.9408,122.5276,76.87,81,95,1.74,ID,2022-01-31 13:47:20
2,Ribeira Grande,38.5167,-28.7000,63.00,59,20,21.85,PT,2022-01-31 13:47:21
3,Hobart,-42.8794,147.3294,67.12,87,40,4.61,AU,2022-01-31 13:46:57
4,Andros Town,24.7000,-77.7667,66.16,52,96,14.12,BS,2022-01-31 13:47:21
5,Salmon Arm,50.6998,-119.3024,37.24,93,100,2.30,CA,2022-01-31 13:47:22
6,Bengkulu,-3.8004,102.2655,78.28,87,100,3.85,ID,2022-01-31 13:42:28
7,Khatanga,71.9667,102.5000,-18.76,100,94,4.27,RU,2022-01-31 13:47:22
8,Saldanha,-33.0117,17.9442,93.25,69,57,15.30,ZA,2022-01-31 13:47:22
9,Vaini,-21.2000,-175.2000,79.57,76,100,12.93,TO,2022-01-31 13:47:23


In [6]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]

city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Albany,US,2022-01-31 13:42:44,42.6001,-73.9662,18.30,72,21,1.74
1,Katobu,ID,2022-01-31 13:47:20,-4.9408,122.5276,76.87,81,95,1.74
2,Ribeira Grande,PT,2022-01-31 13:47:21,38.5167,-28.7000,63.00,59,20,21.85
3,Hobart,AU,2022-01-31 13:46:57,-42.8794,147.3294,67.12,87,40,4.61
4,Andros Town,BS,2022-01-31 13:47:21,24.7000,-77.7667,66.16,52,96,14.12
5,Salmon Arm,CA,2022-01-31 13:47:22,50.6998,-119.3024,37.24,93,100,2.30
6,Bengkulu,ID,2022-01-31 13:42:28,-3.8004,102.2655,78.28,87,100,3.85
7,Khatanga,RU,2022-01-31 13:47:22,71.9667,102.5000,-18.76,100,94,4.27
8,Saldanha,ZA,2022-01-31 13:47:22,-33.0117,17.9442,93.25,69,57,15.30
9,Vaini,TO,2022-01-31 13:47:23,-21.2000,-175.2000,79.57,76,100,12.93


In [7]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")